In [4]:

%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git
%cd /content/Fooocus
!python entry_with_update.py --share --always-high-vram --preset realistic


/content
fatal: destination path 'Fooocus' already exists and is not an empty directory.
/content/Fooocus
Already up-to-date
Update succeeded.
[System ARGV] ['entry_with_update.py', '--share', '--always-high-vram', '--preset', 'realistic']
Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Fooocus version: 2.5.5
Loaded preset: /content/Fooocus/presets/realistic.json
[Cleanup] Attempting to delete content of temp dir /tmp/fooocus
[Cleanup] Cleanup successful
Total VRAM 15095 MB, total RAM 12979 MB
Set vram state to: HIGH_VRAM
Always offload VRAM
Device: cuda:0 Tesla T4 : native
VAE dtype: torch.float32
Using pytorch cross attention
Refiner unloaded.
IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7865
model_type EPS
UNet ADM Dimension 2816
Running on public URL: https://bbd46c098208a25020.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrade

In [8]:
pip install runwayml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 1.9 MB/s eta 0:00:00


In [31]:
import os
import time
import base64 # Import the base64 module
from runwayml import RunwayML

# Configuração do cliente RunwayML
api_key = "SUA_CHAVE_DE_API_AQUI"  # Substitua pela sua chave de API
client = RunwayML(api_key="key_303185cd2597b2871605f9ebf68eb769ce84313c7c40f25ddf39efc3e8ab0561ba489141b7e6d4e146643b32e11365cd8877d92ebe0c7b80805fb1fa133ec800")

# Diretórios de entrada e saída
input_dir = "/content/drive/My Drive/bible_code"
output_video = "/content/drive/My Drive/bible_animation.mp4"
os.makedirs(output_dir, exist_ok=True)

# Função para animar uma imagem
def animate_image(image_path, output_path, prompt_text="Animação da cena."):
    with open(image_path, "rb") as img_file:
        image_data = base64.b64encode(img_file.read()).decode('utf-8')

    # Determine image type for data URI prefix
    image_type = os.path.splitext(image_path)[1][1:].lower()  # Get extension and convert to lowercase
    data_uri_prefix = f"data:image/{image_type};base64,"

    # Criação da tarefa de animação, including data URI prefix
    task = client.image_to_video.create(
        model="gen3a_turbo",
        prompt_image=data_uri_prefix + image_data,  # Add the prefix here
        prompt_text=prompt_text,
    )
    task_id = task.id

    # Aguardar a conclusão da tarefa
    while True:
        task_status = client.tasks.retrieve(task_id)
        if task_status.status in ["SUCCEEDED", "FAILED"]:
            break
        time.sleep(10)  # Verifica a cada 10 segundos

    if task_status.status == "SUCCEEDED":
        # Baixar o vídeo gerado
        video_url = task_status.output.video
        video_data = client._client.get(video_url).content
        with open(output_path, "wb") as video_file:
            video_file.write(video_data)
        print(f"Animação salva em: {output_path}")
    else:
        print(f"Falha ao animar {image_path}")

# Processar todas as imagens na pasta de entrada
for i in range(1, 7):
    img_filename = f"scene_{i}.png"
    img_path = os.path.join(input_dir, img_filename)
    output_path = os.path.join(output_dir, f"scene_{i}_animated.mp4")

    if os.path.exists(img_path):
        animate_image(img_path, output_path)
    else:
        print(f"Imagem não encontrada: {img_path}")

print("Processo de animação concluído.")


BadRequestError: Error code: 400 - {'error': 'You do not have enough credits to run this task.'}

In [26]:
!ls -lh /tmp

total 168K
srwxr-xr-x  1 root root    0 Apr  1 02:59 colab_runtime.sock
-rw-r--r--  1 root root 1.3K Apr  1 02:59 dap_multiplexer.b85cc022e154.root.log.INFO.20250401-025927.114
lrwxrwxrwx  1 root root   62 Apr  1 02:59 dap_multiplexer.INFO -> dap_multiplexer.b85cc022e154.root.log.INFO.20250401-025927.114
srwxr-xr-x  1 root root    0 Apr  1 02:59 debugger_64fz403oa
-rw-r--r--  1 root root 1.4K Apr  1 04:24 directoryprefetcher_binary.b85cc022e154.root.log.INFO.20250401-042405.21344
lrwxrwxrwx  1 root root   75 Apr  1 04:24 directoryprefetcher_binary.INFO -> directoryprefetcher_binary.b85cc022e154.root.log.INFO.20250401-042405.21344
-rw-------  1 root root  749 Apr  1 04:23 drive.b85cc022e154.root.log.ERROR.20250401-042359.21137
-rw-------  1 root root 1.9K Apr  1 04:24 drive.b85cc022e154.root.log.ERROR.20250401-042403.21243
-rw-r--r--  1 root root 2.8K Apr  1 04:23 drive.b85cc022e154.root.log.INFO.20250401-042358.21126
-rw-------  1 root root 3.5K Apr  1 04:23 drive.b85cc022e154.root.log